In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import hopsworks
import pandas as pd
from datetime import timedelta
from src.inference import get_feature_store, fetch_predictions

def fetch_hourly_rides(hours):
    current_hour = (pd.Timestamp.now(tz="Etc/UTC") - timedelta(hours=hours)).floor('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour >= current_hour)

    return query.read()

In [4]:
df = fetch_hourly_rides(12)

2025-03-02 16:36:38,701 INFO: Initializing external client
2025-03-02 16:36:38,701 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-02 16:36:39,894 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213637
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (3.68s) 


In [5]:
df

,pickup_hour,pickup_location_id,rides


In [6]:
df_pred = fetch_predictions(12)

2025-03-02 16:36:46,461 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-02 16:36:46,464 INFO: Initializing external client
2025-03-02 16:36:46,464 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-02 16:36:47,568 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213637
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.55s) 


In [7]:
df_pred

,pickup_location_id,predicted_demand,pickup_hour
0,198,0.0,2025-03-02 04:00:00+00:00
1,239,151.0,2025-03-02 04:00:00+00:00
2,31,0.0,2025-03-02 04:00:00+00:00
3,199,0.0,2025-03-02 04:00:00+00:00
4,30,0.0,2025-03-02 04:00:00+00:00
...,...,...,...
4764,127,0.0,2025-03-02 22:00:00+00:00
4765,184,0.0,2025-03-02 22:00:00+00:00
4766,200,0.0,2025-03-02 22:00:00+00:00
4767,9,0.0,2025-03-02 22:00:00+00:00


In [8]:
merged_df = pd.merge(df, df_pred, on=['pickup_location_id', 'pickup_hour'])

In [9]:
merged_df

,rides,pickup_location_id,predicted_demand,pickup_hour


In [10]:
merged_df['difference'] = merged_df['predicted_demand'] - merged_df['rides']

In [11]:
merged_df.sort_values(["pickup_location_id", "pickup_hour"])

,rides,pickup_location_id,predicted_demand,pickup_hour,difference


In [12]:
merged_df

,rides,pickup_location_id,predicted_demand,pickup_hour,difference


In [13]:
import pandas as pd  
import plotly.express as px
df1 = df
df2 = df_pred

# Merge the DataFrames on 'pickup_location_id' and 'pickup_hour'  
merged_df = pd.merge(df1, df2, on=['pickup_location_id', 'pickup_hour'])  

# Calculate the absolute error  
merged_df['absolute_error'] = abs(merged_df['predicted_demand'] - merged_df['rides'])  

# Group by 'pickup_hour' and calculate the mean absolute error (MAE)  
mae_by_hour = merged_df.groupby('pickup_hour')['absolute_error'].mean().reset_index()  
mae_by_hour.rename(columns={'absolute_error': 'MAE'}, inplace=True)  

# Create a Plotly plot  
fig = px.line(  
    mae_by_hour,  
    x='pickup_hour',  
    y='MAE',  
    title='Mean Absolute Error (MAE) by Pickup Hour',  
    labels={'pickup_hour': 'Pickup Hour', 'MAE': 'Mean Absolute Error'},  
    markers=True  
)  

# Show the plot  
fig.show()

In [14]:
mae_by_hour["MAE"].mean()

nan

In [ ]:
def fetch_hourly_ridess(hours):
    current_hour = (pd.Timestamp.now(tz="Etc/UTC") - timedelta(hours=hours)).floor("h")
    current_hour_str = current_hour.strftime('%Y-%m-%d %H:%M:%S') 
    print("jai balayya")
    print(current_hour)
    print(type(current_hour))
    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_NAME, version=1)

    query = fg.select_all()
    print(query.read())
    print(fg.pickup_hour)
    # query = query.filter(fg.pickup_hour >= current_hour_str)

    return query.read()
df=fetch_hourly_ridess(12)

In [46]:
df=fetch_hourly_ridess(12)
df

jai balayya
2025-03-02 11:00:00+00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2025-03-02 18:52:14,373 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-02 18:52:14,378 INFO: Initializing external client
2025-03-02 18:52:14,378 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-02 18:52:15,141 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213637
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.58s) 
                      pickup_hour  pickup_location_id  rides
0       2025-02-03 15:00:00+00:00                  33      2
1       2025-02-15 14:00:00+00:00                 162    139
2       2025-02-08 15:00:00+00:00                 114     99
3       2025-02-12 09:00:00+00:00                  93      1
4       2025-02-28 23:00:00+00:00                 263    112
...                           ...                 ...    ...
2621400 2023-02-21 08:00:00+

,pickup_hour,pickup_location_id,rides
0,2025-02-03 15:00:00+00:00,33,2
1,2025-02-15 14:00:00+00:00,162,139
2,2025-02-08 15:00:00+00:00,114,99
3,2025-02-12 09:00:00+00:00,93,1
4,2025-02-28 23:00:00+00:00,263,112
...,...,...,...
2621400,2023-02-21 08:00:00+00:00,130,0
2621401,2023-03-16 21:00:00+00:00,232,5
2621402,2023-05-20 01:00:00+00:00,50,43
2621403,2023-08-09 03:00:00+00:00,12,0


In [47]:
current_hour = (pd.Timestamp.now(tz="Etc/UTC") - timedelta(hours=12)).floor("h")
current_hour_str = current_hour.strftime('%Y-%m-%d %H:%M:%S') 
df.sort_values(by=["pickup_hour"], ascending=False)

,pickup_hour,pickup_location_id,rides
1883610,2025-03-01 23:00:00+00:00,186,138
708679,2025-03-01 23:00:00+00:00,50,32
2015529,2025-03-01 23:00:00+00:00,63,0
1296044,2025-03-01 23:00:00+00:00,221,0
333374,2025-03-01 23:00:00+00:00,208,0
...,...,...,...
1078423,2023-01-01 00:00:00+00:00,223,1
858908,2023-01-01 00:00:00+00:00,259,0
664897,2023-01-01 00:00:00+00:00,187,0
1711871,2023-01-01 00:00:00+00:00,235,0
